In [1]:
import numpy as np

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import os

### Model

In [3]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

### Utils

In [4]:
import unidecode
import string
import random
import time
import math
import torch
from torch.autograd import Variable

all_characters = string.printable
n_characters = len(all_characters)

def read_file(filename):
    """Read file"""
    f = unidecode.unidecode(open(filename).read())
    return f, len(f)

def char_tensor(string):
    """String to tensor"""
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)

def elapsed(start):
    """Get elapsed time"""
    secs = time.time() - start
    mins = math.floor(secs / 60)
    secs -= mins * 60
    return '{}m {}s'.format(mins, secs)

def random_chunk(size):
    start_index = random.randint(0, file_len - size)
    end_index = start_index + size + 1
    return file[start_index:end_index]

def random_training_set(size=200, verbose=False):    
    chunk = random_chunk(size)
    if verbose:
        print(chunk)
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [5]:
data_path = os.path.abspath('_data/tiny-shakespeare.txt')

In [6]:
file, file_len = read_file(data_path)
print('file_len =', file_len)
print(file[:100])

file_len = 1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [7]:
random_chunk(200)

"forbid!\n\nGREEN:\nAh, madam, 'tis too true: and that is worse,\nThe Lord Northumberland, his son young Henry Percy,\nThe Lords of Ross, Beaumond, and Willoughby,\nWith all their powerful friends, are fled t"

In [8]:
random_training_set(50)

(tensor([ 15,  34,  94,  29,  17,  14,  14,  73,  96,  36,  23,  13,
          94,  29,  24,  94,  22,  34,  94,  11,  27,  24,  29,  17,
          14,  27,  94,  29,  30,  27,  23,  94,  22,  34,  94,  11,
          21,  30,  28,  17,  18,  23,  16,  94,  12,  17,  14,  14,
          20,  28]),
 tensor([ 34,  94,  29,  17,  14,  14,  73,  96,  36,  23,  13,  94,
          29,  24,  94,  22,  34,  94,  11,  27,  24,  29,  17,  14,
          27,  94,  29,  30,  27,  23,  94,  22,  34,  94,  11,  21,
          30,  28,  17,  18,  23,  16,  94,  12,  17,  14,  14,  20,
          28,  75]))

In [9]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)

    return predicted

In [10]:
def train(inp, target):
    hidden = decoder.init_hidden()
    decoder_optimizer.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden)
        loss += criterion(output, target[c].unsqueeze(0))

    loss.backward()
    decoder_optimizer.step()

    return loss.data.item() / chunk_len

In [12]:
args = {
    "hidden_size": 50,
    "n_layers": 2,
    "lr": 0.005,
    "n_epochs": 2000,
    "print_every": 100,
    "plot_every": 10,
    "hidden_size": 300,
    "chunk_len": 200,
}

n_epochs = args["n_epochs"]
chunk_len = args["chunk_len"]

decoder = RNN(n_characters, args["hidden_size"], n_characters, args["n_layers"])
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=args["lr"])
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    loss = train(*random_training_set(chunk_len))
    loss_avg += loss

    if epoch % args["print_every"] == 0:
        print('[%s (%d %d%%) %.4f]' % (elapsed(start), epoch, epoch / n_epochs * 100, loss))
        print(evaluate('Wh', 100), '\n')

    if epoch % args["plot_every"] == 0:
        all_losses.append(loss_avg / args["plot_every"])
        loss_avg = 0

 58.46490812301636s (100 5%) 2.2629]
Whnarlse
Rond he my gare, with blotus I love, nod I hat und in hat bome here be the nond and dae
I you 

 57.362316846847534s (200 10%) 2.1968]
Whe,
Felp to me deace here your be here with ar wither of this vere free
Meee sord weet.

LOLINE:
Musu 

 54.273353099823s (300 15%) 2.2783]
Where mity not sough,
To desto will
Fuect, beat, I nost we stand sues to and was may well to do of wel 

 52.438225507736206s (400 20%) 2.0721]
Why.

She there freeare my no she full such of for ale word unture:
No say king, a
youming bacce me be 

 51.30783677101135s (500 25%) 1.9114]
Whengoe, buck, affe all-ame to thew forator
The for man there afes and the aw ford the hore the dostil 

 48.33639121055603s (600 30%) 1.8874]
When?

POMEO:
All and this for a geard her thing.

GRUMEO:
Nom marright your you seets a greather hand 

 47.84331774711609s (700 35%) 1.9430]
Whimes pethere a the good?
have brwes over wruhing mink leave act are not they
That mince with theaph  

